In [2]:
from absbox import Generic

test01 = Generic(
    "TEST01- preclosing"
    ,{"cutoff":"2021-03-01","closing":"2021-04-15","firstPay":"2021-07-26","firstCollect":"2021-04-28"
     ,"payFreq":["DayOfMonth",20]
     ,"poolFreq":"MonthEnd"
     ,"stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":30
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,tuple()
    ,{"default":[
         ["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )


In [5]:
from absbox import API
localAPI = API("http://localhost:8081",check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.42.2

In [54]:
import toolz as tz
import re
from lenses import lens

def filterTxn(rs, f, rg):
    return [ r for r in rs if re.match(rg, r[f])] 

In [7]:
r = localAPI.run(test01,read=True)

Warning Message from server:

In [68]:
test01.collection

[['CollectedInterest', 'acc01'],
 ['CollectedPrincipal', 'acc01'],
 ['CollectedPrepayment', 'acc01'],
 ['CollectedRecoveries', 'acc01']]

In [70]:
testSepAcc = tz.pipe( test01 & lens.accounts.modify( lambda xs: tuple(tz.cons( ('acc02',{"balance":0}), xs )))
                      ,lambda d : d & lens.collection[0][1].set("acc02")
                    )

In [33]:
 r['accounts']['acc01'].loc[:'2021-04-15'].to_dict('records')

[{'balance': 28.88, 'change': 28.88, 'memo': '<Pool:CollectedInterest>'},
 {'balance': 162.3, 'change': 133.42, 'memo': '<Pool:CollectedPrincipal>'},
 {'balance': 162.3, 'change': 0.0, 'memo': '<Pool:CollectedPrepayment>'},
 {'balance': 162.3, 'change': 0.0, 'memo': '<Pool:CollectedRecoveries>'}]

In [37]:
filterTxn( r['accounts']['acc01'].loc[:'2021-04-15'].to_dict('records'), 'memo', ".*CollectedInterest.*")[0]['change'] == 28.88

True

In [45]:
r['pool']['flow']['PoolConsol'].loc[:'2021-04-15']["Interest"]

Date
2021-04-15    28.88
Name: Interest, dtype: float64

In [51]:
r['pool']['flow']['PoolConsol'].loc[:'2021-04-15']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-04-15,2066.58,133.42,28.88,0,0,0,0,0.08,None,None,133.42,0,0,0,0,0


In [133]:
test2SepAcc = tz.pipe( test01 & lens.accounts.modify( lambda xs: tuple(tz.cons( ('acc02',{"balance":0}), xs )))
                              & lens.accounts.modify( lambda xs: tuple(tz.cons( ('acc03',{"balance":0}), xs )))
                    ,lambda d : d & lens.collection.set([["Collecte8dInterest","acc02"]
                                                        ,["CollectedPrincipal",[0.7,"acc01"],[0.3,"acc03"]]
                                                        ,["CollectedFeePaid","acc01"]
                                                        ,["CollectedRecoveries","acc01"]]
                        )
                    )

In [134]:
r = localAPI.run(test2SepAcc,read=True)

RuntimeError: not match found: CollectedFe12ePaid :make Pool Source

In [81]:
r['accounts']['acc01'].head()

,balance,change,memo
date,,,
2021-04-15,93.39,93.39,<Pool:CollectedPrincipal>
2021-04-15,93.39,0.00,<Pool:CollectedPrepayment>
2021-04-15,93.39,0.00,<Pool:CollectedRecoveries>
2021-04-30,93.39,0.00,<Pool:CollectedPrincipal>
2021-04-30,93.39,0.00,<Pool:CollectedPrepayment>


In [82]:
r['accounts']['acc02'].head()

,balance,change,memo
date,,,
2021-04-15,28.88,28.88,<Pool:CollectedInterest>
2021-04-30,28.88,0.00,<Pool:CollectedInterest>
2021-05-31,42.65,13.77,<Pool:CollectedInterest>
2021-06-30,55.97,13.32,<Pool:CollectedInterest>
2021-07-31,68.84,12.87,<Pool:CollectedInterest>


In [83]:
r['accounts']['acc03'].head()

,balance,change,memo
date,,,
2021-04-15,40.02,40.02,<Pool:CollectedPrincipal>
2021-04-30,40.02,0.00,<Pool:CollectedPrincipal>
2021-05-31,60.23,20.21,<Pool:CollectedPrincipal>
2021-06-30,80.57,20.34,<Pool:CollectedPrincipal>
2021-07-31,101.05,20.48,<Pool:CollectedPrincipal>


In [87]:
40.02 + 93.39

133.41

In [88]:
133.42 

133.42

In [84]:
r['pool']['flow']['PoolConsol'].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-04-15,2066.58,133.42,28.88,0,0,0,0,0.08,None,None,133.42,0,0,0,0,0
2021-04-30,2066.58,0.00,0.00,0,0,0,0,0.08,None,None,133.42,0,0,0,0,0
2021-05-31,1999.20,67.38,13.77,0,0,0,0,0.08,None,None,200.80,0,0,0,0,0
2021-06-30,1931.37,67.83,13.32,0,0,0,0,0.08,None,None,268.63,0,0,0,0,0
2021-07-31,1863.09,68.28,12.87,0,0,0,0,0.08,None,None,336.91,0,0,0,0,0


In [90]:
test01 & lens.pool.get()

{'assets': [['Mortgage',
   {'originBalance': 2200,
    'originRate': ['fix', 0.045],
    'originTerm': 30,
    'freq': 'Monthly',
    'type': 'Level',
    'originDate': '2021-02-01'},
   {'currentBalance': 2200,
    'currentRate': 0.08,
    'remainTerm': 30,
    'status': 'current'}]]}

In [116]:
testMultiplePool = test01 & lens.accounts.set((("acc01",{"balance":0}),("acc02",{"balance":0}),("acc03",{"balance":0}))) \
                          & lens.pool.set({"PoolA":{'assets':[["Mortgage"
                                                      ,{"originBalance":1320.0,"originRate":["fix",0.045],"originTerm":30
                                                        ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                                                        ,{"currentBalance":1320.0
                                                        ,"currentRate":0.08
                                                        ,"remainTerm":30
                                                        ,"status":"current"}]]},
                                            "PoolB":{'assets':[["Mortgage"
                                                      ,{"originBalance":880.0,"originRate":["fix",0.045],"originTerm":30
                                                        ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                                                        ,{"currentBalance":880.0
                                                        ,"currentRate":0.08
                                                        ,"remainTerm":30
                                                        ,"status":"current"}]]}}) \
                          & lens.collection.set([[["PoolA",],"CollectedInterest","acc01"]
                                                 ,[["PoolB",],"CollectedInterest","acc03"]
                                                 ,[["PoolA","PoolB"],"CollectedPrincipal",[0.7,"acc02"],[0.3,"acc03"]]
                                                ]
                                               )

In [117]:
r = localAPI.run(testMultiplePool,read=True)

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off
Account acc02 has cash to be distributed
Account acc03 has cash to be distributed

In [121]:
r['pool']['flow']['PoolA'].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-04-15,1239.94,80.06,17.32,0,0,0,0,0.08,None,None,80.06,0,0,0,0,0
2021-04-30,1239.94,0.00,0.00,0,0,0,0,0.08,None,None,80.06,0,0,0,0,0
2021-05-31,1199.51,40.43,8.26,0,0,0,0,0.08,None,None,120.49,0,0,0,0,0
2021-06-30,1158.81,40.70,7.99,0,0,0,0,0.08,None,None,161.19,0,0,0,0,0
2021-07-31,1117.84,40.97,7.72,0,0,0,0,0.08,None,None,202.16,0,0,0,0,0


In [119]:
r['pool']['flow']['PoolB'].head(2)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-04-15,826.62,53.38,11.54,0,0,0,0,0.08,None,None,53.38,0,0,0,0,0
2021-04-30,826.62,0.00,0.00,0,0,0,0,0.08,None,None,53.38,0,0,0,0,0


In [126]:
(53.38+80.06) * 0.7,(53.38+80.06) * 0.3

(93.40799999999999, 40.032)

In [120]:
r['accounts']['acc01'].head()

,balance,change,memo
date,,,
2021-04-15,17.32,17.32,<PoolPoolA:CollectedInterest>
2021-04-30,17.32,0.00,<PoolPoolA:CollectedInterest>
2021-05-31,25.58,8.26,<PoolPoolA:CollectedInterest>
2021-06-30,33.57,7.99,<PoolPoolA:CollectedInterest>
2021-07-26,14.01,-19.56,<PayInt:A1>


In [122]:
r['accounts']['acc02'].head()

,balance,change,memo
date,,,
2021-04-15,93.40,93.40,<PoolPoolA|PoolB:CollectedPrincipal>
2021-04-30,93.40,0.00,<PoolPoolA|PoolB:CollectedPrincipal>
2021-05-31,140.56,47.16,<PoolPoolA|PoolB:CollectedPrincipal>
2021-06-30,188.04,47.48,<PoolPoolA|PoolB:CollectedPrincipal>
2021-07-31,235.84,47.80,<PoolPoolA|PoolB:CollectedPrincipal>


In [123]:
r['accounts']['acc03'].head()

,balance,change,memo
date,,,
2021-04-15,11.54,11.54,<PoolPoolB:CollectedInterest>
2021-04-15,51.57,40.03,<PoolPoolA|PoolB:CollectedPrincipal>
2021-04-30,51.57,0.00,<PoolPoolB:CollectedInterest>
2021-04-30,51.57,0.00,<PoolPoolA|PoolB:CollectedPrincipal>
2021-05-31,57.08,5.51,<PoolPoolB:CollectedInterest>
